# Vectorize spell checking engine

### Description
Implementation of a fast spell checking and suggestion engine using character vectors.

### Methodology:
- Acquire a words list that will serve as the absolute truth of spellings.
- Use tfidf or phoenetic vectorization method to encode words.
- Use similarity metrics to identify possible respelling of out of vocabulary terms in a document.

### Word list resources

- http://wordlist.aspell.net/
- https://github.com/dwyl/english-words :: https://github.com/dwyl/english-words/blob/master/words_dictionary.json

In [1]:
# -*- coding: utf-8 -*-

In [2]:
%load_ext autotime

In [1]:
import os
import re
import numpy as np
import pandas as pd
import nltk
import json

from nltk.corpus import words
from joblib import Parallel, delayed
import multiprocessing as mp
import multiprocessing

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.metrics.distance import edit_distance

from scipy.stats import rankdata

import spacy

from contexttimer import Timer
import requests

In [4]:
from lsh import minhash, cache

time: 6.43 ms


In [4]:
from sklearn.decomposition import TruncatedSVD

time: 7.3 ms


In [5]:
from nltk import word_tokenize

time: 1.73 ms


In [21]:
from enchant.checker import SpellChecker
from enchant import Dict

time: 6.87 ms


In [22]:
class OptimizedSpellChecker(SpellChecker):
    '''
    Reduces the tokens only to unique words in the text. Output is not in the same order relative
    to the original text.
    '''
    dict_words = set()
    
    def __init__(self, lang=None, text=None, tokenize=None, chunkers=None, filters=None):
        super().__init__(
            lang=lang, text=text, tokenize=tokenize, chunkers=chunkers, filters=filters
        )
        
    def set_tokens(self, tokens):
        """Set the text to be spell-checked.
        This method must be called, or the 'text' argument supplied
        to the constructor, before calling the 'next()' method.
        """
        self._tokens = enumerate(tokens)
        
    def next(self):
        """Process text up to the next spelling error.
        
        This method is designed to support the iterator protocol.
        Each time it is called, it will advance the 'word' attribute
        to the next spelling error in the text.  When no more errors
        are found, it will raise StopIteration.
        
        The method will always return self, so that it can be used
        sensibly in common idioms such as:
            for err in checker:
                err.do_something()
        
        """
        # Find the next spelling error.
        # The uncaught StopIteration from next(self._tokens)
        # will provide the StopIteration for this method
        while True:
            pos, word = next(self._tokens)
            if word in self.dict_words:
                continue
            if self.dict.check(word):
                self.dict_words.add(word)
                continue
            if word in self._ignore_words:
                continue
            self.word = word
            self.wordpos = pos
            if word in self._replace_words:
                self.replace(self._replace_words[word])
                continue
            break
        return self


time: 77.6 ms


In [23]:
en_dict = Dict(u'en_US')
spellchecker = OptimizedSpellChecker(u"en_US")

time: 14.4 ms


In [8]:
%time en_dict.suggest(u'nighttime')

CPU times: user 32.7 ms, sys: 319 µs, total: 33 ms
Wall time: 32.2 ms


['nighttime',
 'nighttimes',
 'night time',
 'night-time',
 'nightmare',
 'nightmarish']

time: 40.2 ms


![im](https://cdn-images-1.medium.com/max/1600/1*5bvgtaEv3UcowA3vw8SWpQ.png)

In [6]:
import itertools

time: 1.71 ms


In [7]:
with open('./data/words.txt') as fl:
    words = dict(enumerate([i.strip() for i in fl.readlines() if i.strip().isalpha()]))

print(len(words))

416297
time: 142 ms


In [15]:
hasher = minhash.MinHasher(seeds=100, char_ngram=3, hashbytes=4)
lsh_cache = cache.Cache(bands=10, hasher=hasher)

for word_id, word in words.items():
    lsh_cache.add_fingerprint(hasher.fingerprint(word), word_id)

duplicate_pairs = set()

total_bins = 0

for cache_bin in lsh_cache.bins:
    for bucket_id in cache_bin:
        if len(cache_bin[bucket_id]) > 1:
            pairs = set(itertools.combinations(cache_bin[bucket_id], r=2))
            duplicate_pairs.update(pairs)
        total_bins += 1

duplicate_pairs = list(duplicate_pairs)
total_bins

time: 24.8 s


In [16]:
total_bins

3903171

time: 1.13 ms


In [14]:
[(words[i], words[j]) for i, j in duplicate_pairs[:10]]

[('unassumable', 'assumable'),
 ('ji', 'RL'),
 ('geosphere', 'cosphered'),
 ('Jr', 'FB'),
 ('reenlistness', 'reenlistnesses'),
 ('undispose', 'undisposed'),
 ('anticlassicism', 'anticlassicist'),
 ('Verrucaria', 'Verrucariaceae'),
 ('le', 'WH'),
 ('mischaracterize', 'mischaracterized')]

time: 3.53 ms


In [17]:
swords = set(words.values())

time: 23.4 ms


In [18]:
'Python' in swords

True

time: 3.01 ms


In [10]:
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3,3), dtype=np.float32)
tsvd = TruncatedSVD(n_components=50)

dict_words_vecs = tfidf.fit_transform(words)
dict_words_vecs = tsvd.fit_transform(dict_words_vecs)

time: 6.37 s


In [11]:
w = 'I pulled out the words into a simple new-line-delimited text file. Which is more useful when building apps or importing into databases etc.'
tokens = word_tokenize(w)

time: 5.29 ms


In [20]:
def load_text(doc_name='11758940.txt', path='/home/avsolatorio/WBG/NLP/WB/CORPUS/RAW/eap_files'):
    fname = os.path.join(path, doc_name)
    
    with open(fname, 'rb') as fl:
        text = fl.read()
        text = u'' + text.decode('utf-8', errors='ignore')
    
    return text

time: 2.84 ms


In [22]:
text = load_text()
tokens = word_tokenize(text)

time: 11.5 s


In [32]:
spellchecker.set_tokens(tokens)
errors = set()

for err in spellchecker:
    errors.add(err.word)
    
errors = list(errors)

time: 1.71 s


In [33]:
doc = tsvd.transform(tfidf.transform(errors))

time: 49.8 ms


In [34]:
dict_words_vecs.shape, doc.shape

((466551, 50), (13654, 50))

time: 3.35 ms


In [35]:
sims = cosine_similarity(doc, dict_words_vecs)

time: 11.1 s


In [36]:
gg = list(zip([words[i] for i in sims.argmax(axis=1)], sims[:,sims.argmax(axis=1)].max(axis=1)))

time: 14 s


In [47]:
errors[60]

'Buu'

time: 3.29 ms


In [43]:
list(enumerate(gg[:100]))

[(0, ('2', 0.0)),
 (1, ('Dilan', 0.9957872)),
 (2, ('2', 0.0)),
 (3, ('2', 0.0)),
 (4, ('Bai', 0.9142381)),
 (5, ('pocill', 0.99707234)),
 (6, ('2', 0.0)),
 (7, ('2', 0.0)),
 (8, ('2', 0.0)),
 (9, ('2', 0.0)),
 (10, ('2', 0.0)),
 (11, ('vill', 0.99930024)),
 (12, ('2', 0.0)),
 (13, ('2', 0.0)),
 (14, ('DAY', 0.9958306)),
 (15, ('2', 0.0)),
 (16, ('Y.M.C.A.', 0.9921591)),
 (17, ('2', 0.0)),
 (18, ('2', 0.0)),
 (19, ('2', 0.0)),
 (20, ('moggy', 0.95128155)),
 (21, ('2', 0.0)),
 (22, ('thiazides', 0.9641419)),
 (23, ('2', 0.0)),
 (24, ('trangam', 0.997847)),
 (25, ('2', 0.0)),
 (26, ("l'tre", 0.97923344)),
 (27, ('gung-ho', 0.84768647)),
 (28, ('kilhig', 0.7925369)),
 (29, ('servilities', 0.9794461)),
 (30, ('2', 0.0)),
 (31, ('Kim', 0.9884073)),
 (32, ('TRH', 0.9949274)),
 (33, ('2', 0.0)),
 (34, ('2', 0.0)),
 (35, ('2', 0.0)),
 (36, ('2', 0.0)),
 (37, ('2', 0.0)),
 (38, ('2', 0.0)),
 (39, ('2', 0.0)),
 (40, ('2', 0.0)),
 (41, ('2', 0.0)),
 (42, ('2', 0.0)),
 (43, ('workingwoman', 0.8699

time: 7.44 ms


In [2]:
words_data = pd.read_json('./data/words_dictionary.json', typ='ser')

In [7]:
words_data['philippines']

1

In [10]:
"Ss s".isalpha()

False

In [9]:
w[:1000]

b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-3+HOqCwtQal5hOJQ+mdxiq5zmGOTjF6RhjDsPLxbKDYgGlLFeCwzoIanb7j5IiCuXKUqyC2q8FdkC4nmx2P2rA==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-a2fba223d5af91496cac70d4ec3624df.css" />\n  <link crossorigin="anonymous" media="all" integrity="sha512-4ohd09bNnMlKWClfY22ZwyWNN3GJm6wo4ooCUoXfNecOKxjerWekbqlOj